## Importing the libraries

In [1]:
import os
import copy
import math
import random
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics.cluster import contingency_matrix
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extracting the features

In [2]:
featuresPath = "/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/features.txt"
with open(featuresPath, "r") as file:
  # splitting at ':' and taking the first word
  features = [line.split(':')[0] for line in file]
# dropping the first 2 lines as they are unnecessary
features.pop(0), features.pop(0)
features.append('attack_type')

In [3]:
len(features)

41

## Reading the dataset

In [4]:
path = "/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/kddcup.data_10_percent.gz"
df = pd.read_csv(path, compression='gzip', names=features, header=None)
df.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,tcp,http,SF,181,5450,0,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,239,486,0,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,235,1337,0,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,219,1337,0,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,217,2032,0,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [5]:
# Size of train Set
print(df.shape)

(494021, 41)


In [6]:
path = "/content/drive/MyDrive/Colab Notebooks/Anomaly Detection/corrected.gz"
df_test = pd.read_csv(path, compression='gzip', names=features, header=None)
df_test.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,udp,private,SF,105,146,0,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
0,udp,private,SF,105,146,0,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
0,udp,private,SF,105,146,0,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
0,udp,private,SF,105,146,0,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
0,udp,private,SF,105,146,0,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


In [7]:
# To concatenate train and test sets
vertical_concat = pd.concat([df, df_test], axis=0)

In [8]:
# getting all categorical columns
cat_columns = df_test.select_dtypes(['object']).columns
# converting all categorical columns to numeric
vertical_concat[cat_columns] = vertical_concat[cat_columns].apply(lambda x: pd.factorize(x)[0])
vertical_concat.tail()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type
0,1,11,0,105,147,0,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0
0,1,11,0,105,147,0,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0
0,1,11,0,105,147,0,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0
0,1,11,0,105,147,0,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0
0,1,11,0,105,147,0,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0


In [9]:
# splitting the dataset again after encoding
X_train = vertical_concat.iloc[:len(df),:]
X_test = vertical_concat.iloc[len(df):,:]

In [10]:
print(vertical_concat['attack_type'].unique())
print(X_train['attack_type'].unique())
print(X_test['attack_type'].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
[ 0 23 24 25  5 10 18 26 27  6 28  1  9  7 29 16 30 19  3 15 31 32 33 34
 17 22  4  2 14 13 35 36 37 12  8 11 38 39]


### Dropping the duplicates

In [11]:
print(f"Shape of train set before dropping duplicates: {len(X_train)}")
print(f"Shape of test set before dropping duplicates: {len(X_test)}")
X_train = X_train.drop_duplicates()
X_test = X_test.drop_duplicates()
print(f"Shape of train set after dropping duplicates: {len(X_train)}")
print(f"Shape of test set after dropping duplicates: {len(X_test)}")
labels_train = X_train['attack_type']
labels_test = X_test['attack_type']

Shape of train set before dropping duplicates: 494021
Shape of test set before dropping duplicates: 311029
Shape of train set after dropping duplicates: 143671
Shape of test set after dropping duplicates: 77084


In [12]:
X_train = X_train.iloc[:,:-1].to_numpy()
X_test = X_test.iloc[:,:-1].to_numpy()
labels_train = labels_train.to_numpy()
labels_test = labels_test.to_numpy()

In [13]:
print(X_train.shape)
print(X_test.shape)

(143671, 40)
(77084, 40)


## KMeans Clustering

In [14]:
class KMeansClustering():
   def fit(self, X, Y, k, epsilon=0, maxIter=20):
       self.pred = []
       # initializing the error to be max at first
       self.err = float('inf')  

       # randomly initializing the centroids
       self.centroid = [X[np.random.randint(0, X.shape[0])] for i in range(k)]
       while maxIter > 0 and self.err > epsilon:
          maxIter -= 1
          self.cluster = [[] for i in range(k)]
          self.cluster_truth = [[] for i in range(k)]
          self.pred.clear()
          # copying the old centroids
          self.oldCentroid = copy.deepcopy(self.centroid)

          # assigning each data point to closest cluster
          y=0
          for x in X:
            minn, idx_min = float('inf'), 0
            for i in range(k):
              dist = np.linalg.norm(x-self.centroid[i]) ** 2
              if dist < minn: minn, idx_min = dist, i
            self.cluster[idx_min].append(x)
            self.cluster_truth[idx_min].append(Y[y])
            self.pred.append(idx_min)
            y = y+1

          # updating the centroids
          for i in range(len(self.cluster)):
            if len(self.cluster[i]) != 0: self.centroid[i] = np.mean(self.cluster[i], axis = 0)

          # if centroids didn't change, then terminate
          self.err = 0
          for i in range(len(self.centroid)):
           self.err += np.linalg.norm(self.centroid[i] - self.oldCentroid[i]) ** 2

       return self.pred , self.cluster_truth, self.centroid

   def predict(self, X_test):
     predictions = []
     for t in range(len(X_test)):   
      dist = [np.linalg.norm(X_test[t]-self.centroid[i]) for i in range(k)]
      # prediction is the one with the closest distance
      predictions.append(np.argmin(dist)) 
     return predictions

In [15]:
# To remove labels in X_test that are not in X_train
train_unique_labels = np.unique(labels_train)
test_unique_labels = np.unique(labels_test)
b = [i for i in test_unique_labels if i not in train_unique_labels]

for i in range(len(b)):
   X_test = np.delete(X_test, np.where(labels_test == b[i]), axis = 0)
   labels_test = np.delete(labels_test, np.where(labels_test == b[i]), axis = 0)
print(X_train.shape)
print(X_test.shape)

(143671, 40)
(73122, 40)


In [16]:
def purity(y, y_pred):
  contingency_matrixx = contingency_matrix(y, y_pred)
  num_total_elements = np.sum(contingency_matrixx)
  prec_total = 0
  for i in range (contingency_matrixx.shape[0]):
    idxmax = np.argmax(contingency_matrixx[i])
    num_elements_per_cluster = np.sum(contingency_matrixx[i])
    prec = contingency_matrixx[i][idxmax] / num_elements_per_cluster
    purity = num_elements_per_cluster / num_total_elements 
    prec_total += np.sum(np.multiply(prec , purity))
  return prec_total

In [148]:
def Fscore(y, y_pred):
  contingency_matrixx = contingency_matrix(y, y_pred)
  F = np.zeros((contingency_matrixx.shape[0], 1))
  num_total_elements = np.sum(contingency_matrixx)
  for i in range (contingency_matrixx.shape[0]):
    idxmax = np.argmax(contingency_matrixx[i])
    num_elements_per_cluster = np.sum(contingency_matrixx[i])
    prec = contingency_matrixx[i][idxmax] / num_elements_per_cluster
    recall = contingency_matrixx[i][idxmax] / np.sum(contingency_matrixx[:, idxmax])
    F[i] = 2 * (prec * recall) / (prec + recall)
  fmeasure = np.sum(F) / F.shape[0]
  return fmeasure

In [18]:
def entropy(y, y_pred):
  contingency_matrixx = contingency_matrix(y, y_pred)
  num_total_elements = np.sum(contingency_matrixx)
  Entro = np.zeros((contingency_matrixx.shape[0],1))
  for i in range (contingency_matrixx.shape[0]):
    num_elements_per_cluster= np.sum(contingency_matrixx[i])
    Entr = num_elements_per_cluster / num_total_elements
    Entro[i] = -1 * math.log(Entr, 2) * Entr
  Entropy =  np.sum(Entro) / Entro.shape[0]
  return Entropy

In [ ]:
KMC = KMeansClustering()
p, clustruth, cent = KMC.fit(X_train, labels_train, k=7)

print("Fmeasure",Fscore(p, labels_train))
print("Entropy",entropy(p, labels_train))
print("Precision", purity(p, labels_train))

Fmeasure 0.37450096995916304
Entropy 0.02442695312731772
Precision 0.6044295647695082


In [ ]:
KMC = KMeansClustering()
p, clustruth, cent = KMC.fit(X_train, labels_train, k=15)

print("Fmeasure",Fscore(p, labels_train))
print("Entropy",entropy(p, labels_train))
print("Precision", purity(p, labels_train))

Fmeasure 0.31409308577262324
Entropy 0.14533441277275386
Precision 0.8597768512782675


In [ ]:
KMC = KMeansClustering()
p, clustruth, cent = KMC.fit(X_train, labels_train, k=23)

print("Fmeasure",Fscore(p, labels_train))
print("Entropy",entropy(p, labels_train))
print("Precision", purity(p, labels_train))

Fmeasure 0.22153746898910964
Entropy 0.16246354165441276
Precision 0.960722762422479


In [ ]:
KMC = KMeansClustering()
p, clustruth, cent = KMC.fit(X_train, labels_train, k=31)

print("Fmeasure",Fscore(p, labels_train))
print("Entropy",entropy(p, labels_train))
print("Precision", purity(p, labels_train))

Fmeasure 0.19410177011133953
Entropy 0.13598904145476218
Precision 0.9615649643978256


In [ ]:
KMC = KMeansClustering()
p, clustruth, cent = KMC.fit(X_train, labels_train, k=45)

print("Fmeasure",Fscore(p, labels_train))
print("Entropy",entropy(p, labels_train))
print("Precision", purity(p, labels_train))

Fmeasure 0.17028703926140817
Entropy 0.1101438585744588
Precision 0.9689498924626401


In [19]:
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_train, labels_train, train_size=0.005, random_state=42, stratify=labels_train)

In [20]:
np.unique(y_train_red)

array([ 0,  4,  5,  7,  8,  9, 10, 13, 15, 17, 20])

In [21]:
def normcut(F,k,gamma,kernel = True):
    if kernel == True:
        kern = rbf_kernel(F, gamma=gamma)
    else:
        kern = kneighbors_graph(F,k).toarray()
    kern = np.array(kern)
    degree = np.zeros(len(kern))
    c_sum = kern.sum(axis = 0)
    r_sum = kern.sum(axis = 1)
    for i in range(c_sum.shape[0]):
        degree[i] = (c_sum[i] + r_sum[i])/2
    degree = np.diag(degree)
    kern = degree - kern
    degree = np.linalg.inv(degree)
    kern = np.matmul(degree,kern)
    (evalues,evectors) = np.linalg.eig(kern)
    idx = evalues.argsort()
    evalues = [idx]
    evectors = evectors[ : , idx]
    evectors = evectors.T[:k]
    normFactor = np.sqrt(np.sum(np.power(evectors,2),axis=1))
    normFactor = normFactor.reshape((len(evectors),1))
    for i in range(len(normFactor)):
        if normFactor[i] == 0:
            normFactor[i] = 1
    Y = np.real(evectors/normFactor).T
    return Y, KMeans(n_clusters=k).fit(Y).labels_

In [144]:
(Y_h , labels_h) = normcut(X_train_red, 23, 0.01, kernel = False)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [149]:
print("Fmeasure",Fscore(labels_h, y_train_red))
print("Entropy",entropy(labels_h, y_train_red))
print("Precision", purity(labels_h, y_train_red))

Fmeasure 0.24158958050478208
Entropy 0.16317821287475448
Precision 0.9637883008356547


## New Clustering Algorithm (Agglomerative Hierarchical Clustering)

In [134]:
def isThere(arr, listOfArr):
  if arr is None: return False
  for arr2 in listOfArr:
    if np.array_equal(arr.ravel(), arr2.ravel()): return True
  return False

In [135]:
def convertClusterToLabels(X, clusters):
  l = []
  for point in X: 
    for i in range(len(clusters)):
      if isThere(point, clusters[i]): l.append(i)
  return l

### Original algorithm

In [22]:
def computeDistance(cluster1, cluster2):
  dist = 0
  for c1 in cluster1:
    for c2 in cluster2:
      dist += np.linalg.norm(np.array(c1)-np.array(c2))
  return dist

In [23]:
def computeProximityMatrix(clusters):
  proxMat = np.zeros((len(clusters), len(clusters)), dtype=np.int8)
  for i in range(len(clusters)):
    print(i)
    for j in range(len(clusters)):
      proxMat[i, j] = computeDistance(clusters[i], clusters[j])
  return proxMat

In [24]:
def updateProximityMatrix(distance, clusters, ij, i, j, type="ward"):
  for r in range(len(distance)-1):
    if r != i and r != j:
      # ward calculations
      if type == "ward":
        ni, nj, nr = len(clusters[i]), len(clusters[j]), 0
        for cluster in clusters: nr += len(cluster)
        alphai = (ni + nr) / (ni + nj + nr)
        alphaj = (nj + nr) / (ni + nj + nr)
        beta = -nr / (ni + nj + nr)
        gamma = 0
      # single link caluclations
      else: alphai, alphaj, beta, gamma = 0.5, 0.5, 0, -0.5
      distance[ij, r] = alphai*distance[i, r] + alphaj*distance[j, r] + beta*distance[i, j] + gamma*abs(distance[i, r] - distance[j, r])
      distance[r, ij] = alphai*distance[r, i] + alphaj*distance[r, j] + beta*distance[j, i] + gamma*abs(distance[r, i] - distance[r, j])
  distance = np.delete(distance, i, 0)
  distance = np.delete(distance, j-1, 0)
  distance = np.delete(distance, i, 1)
  distance = np.delete(distance, j-1, 1)
  return distance

In [132]:
def agglomerativeClustering(D, k):
  # creating a cluster for each point
  clusters = [[D[i]] for i in range(len(D))]
  # computing the distance matrix
  # distance = computeProximityMatrix(clusters)
  distance = pairwise_distances(D)
  while len(clusters) > k:
    # initializing big values to diagonal so that it won't be taken
    distance[np.diag_indices_from(distance)] = float('inf')
    # finding the closest pair of clusters, that are non zeros
    idx = np.where(distance == distance.min())
    i, j = idx[0][0], idx[1][0]
    if i == j: return clusters
    newCluster = np.vstack((clusters[i], clusters[j]))
    # maintain their order while poping
    i, j = (i, j) if i < j else (j, i)
    clusters.pop(i)
    clusters.pop(j-1)
    clusters.append(newCluster)
    distance = np.append(distance, np.zeros((1, len(distance))), axis=0)
    distance = np.append(distance, np.zeros((len(distance), 1)), axis=1)
    # updating the distance matrix
    distance = updateProximityMatrix(distance, clusters, len(distance)-1, i, j, type="ward")
  return clusters

In [133]:
clusters = agglomerativeClustering(X_train_red, 23)

In [136]:
p = convertClusterToLabels(X_train_red, clusters)

print("Fmeasure",Fscore(p, y_train_red))
print("Entropy",entropy(p, y_train_red))
print("Precision", purity(p, y_train_red))

Fmeasure 0.18147843300119568
Entropy 0.17527092498195668
Precision 0.9484679665738162


## New Clustering Algorithm (Mini-batch Kmeans Clustering)

#### But the previous algorithm takes too much space to compute the distance matrix, so we will use a simpler faster algorithm like minibatch kmeans

In [74]:
def createBatchDataset(D, batch_size):
  indices = random.sample(range(0, len(D)-1), batch_size)
  return D[indices]

In [103]:
def updateStep(M, cluster, centroid):
  # assigning each data point to closest cluster
  for x in M:
    minn, j = float('inf'), 0
    for i in range(len(centroid)):
      dist = np.linalg.norm(x-centroid[i]) ** 2
      if dist < minn: minn, j = dist, i
    cluster[j].append(x)
  # updating the centroids
  for i in range(len(cluster)):
    if len(cluster[i]) != 0: centroid[i] = sum(cluster[i]) / len(cluster[i])
  return cluster, centroid

In [104]:
def minibatchKMC(X, k, batch_size, epsilon=0.001, maxIter=1000):
  # initializing the error to be max at first
  err = float('inf')
  # randomly initializing the centroids
  centroid = [X[np.random.randint(0, len(X)-1)] for i in range(k)]
  while maxIter > 0 and err >= epsilon:
    maxIter -= 1
    cluster = [[] for i in range(k)]
    # copying the old centroids
    oldCentroid = copy.deepcopy(centroid)
    # creating a batch dataset
    batch_dataset = createBatchDataset(X, batch_size)
    # updating clusters and centroids
    cluster, centroid = updateStep(batch_dataset, cluster, centroid)
    # if centroids didn't change, then terminate
    err = 0
    for i in range(len(centroid)):
      err += np.linalg.norm(centroid[i] - oldCentroid[i]) ** 2
  # updating clusters and centroids one last time on full data
  cluster = [[] for i in range(k)]
  cluster, centroid = updateStep(X, cluster, centroid)
  return cluster

In [141]:
def Fscore(y, y_pred):
  contingency_matrixx = contingency_matrix(y, y_pred)
  F = np.zeros((contingency_matrixx.shape[0], 1))
  num_total_elements = np.sum(contingency_matrixx)
  for i in range (contingency_matrixx.shape[0]):
    idxmax = np.argmax(contingency_matrixx[i])
    num_elements_per_cluster = np.sum(contingency_matrixx[i])
    prec = contingency_matrixx[i][idxmax] / num_elements_per_cluster
    recall = contingency_matrixx[i][idxmax] / np.sum(contingency_matrixx[:, idxmax])
    F[i] = 2 * (prec * recall) / (prec + recall)
  fmeasure = np.sum(F) / F.shape[0]
  return fmeasure, prec, recall

In [139]:
clusters = minibatchKMC(X_train_red, k=23, batch_size=100, maxIter=100)

In [140]:
p = convertClusterToLabels(X_train_red, clusters)

print("Fmeasure",Fscore(p, y_train_red))
print("Entropy",entropy(p, y_train_red))
print("Precision", purity(p, y_train_red))

Fmeasure 0.20460033187338092
Entropy 0.14709848516778037
Precision 0.923398328690808


In [143]:
fm, prec, rec = Fscore(p, y_train_red)
print(fm, prec, rec)

0.20460033187338092 1.0 0.0069767441860465115
